### VGG16 in Tensorflow
#### Very Deep Convolutional Networks for Large Scale Recognition 
Karen Simonyan & Andrew Zisserman
https://arxiv.org/pdf/1409.1556.pdf

- evaluation of networks of increasing depth with very small (3×3) convolution filters, small enough to capture the notion of left/right, up/down, center
- *simple* architecture 
    - input: 224x224x3
    - stacks of convolutional layers with kernel 3x3/stride 1, relu activation, width of 64,128,256,512
    - max pooling kernel 2x2/stride 2
    - 3 fully connected 4096
- 5 configurations were tested (from 8conv/3fc layers to 16 conv/3fc layers)
    - VGG-A: 11weight layers -- 133M params -- top1 29.6% / top5 10.4%
    - VGG-B: 13wl -- 133M -- top1 28.7% top5 9.9%
    - VGG-C: 16wl -- 134M -- top1 28.1% top5 9.4%
    - VGG-D: 16wl -- 138M -- top1 27.0% top5 8.8%
    - VGG-E: 19wl -- 144M -- top1 27.3% top5 8.0%
    - more layers are feasible because of the small convolution filters
- training
    - mini-batch gradient descent (batch size 256) with momentum (0.9)
    - weight decay 0.0005
        - regularizer, because of its small ammount it reduces the model’s training error too (AlexNet paper).
    - dropout 0.5 for first two fully connected layers
    - learning rate 0.01, 0.001, 0.0001, 0.00001
    - 74 epochs, less epochs to converge because:
        - regularization imposed greater depth and smaller conv filters
        - preinitialization - very important
            - shallow configurations (configuration A) were initialized randomly
            - deeper configuration were initialized with layers of A (intermediate layers were randomly initialized - zero man 0.01 variance, bias 0)
            - there are better ways to initialize layers
    - augmentation
        - random horizontal flipping
        - random rgb color shifting
        - scale jittering, which improves the performance
- implementation
    - c++/caffe
    - gpu parallelization
        - model & data parallelism for different layers
    - 2-3 week of training in a 4-gpu system (nvidia titan black gpu)
- dataset ILSVRC-2012
    - 1000 classes
        - 1.3M images training
        - 50K validation
        - 100K testing
- ensemble of D&E configurations
    - *best result is achieved by combining just two models*: significantly less than used in most ILSVRC submissions.


#### 1. Download the weights and working files
- https://www.cs.toronto.edu/~frossard/post/vgg16/

#### 2. Build the class

In [1]:
import tensorflow as tf
import numpy as np
from scipy.misc import imread, imresize

In [2]:
class vgg16:
    def __init__(self, imgs, weights=None, sess=None):
        self.imgs = imgs

        # lista que almacenará elementos (capas, bias)
        # para facilitar la inicialización de pesos luego
        self.parameters = None

        # construimos las capas según la arquitectura descrita en el paper
        # capas convolucionales
        self.convlayers()
        # capas densas
        self.fc_layers()
        # activación final
        self.probs = tf.nn.softmax(self.fc3l)
        
        if weights is not None and sess is not None:
            self.load_weights(weights, sess)


    #
    # Capas convolucionales
    #
    def convlayers(self):
        self.parameters = []

        #
        # Construiremos la configuración VGG-D
        #

        # Preprocesamiento: restamos la media a cada pixel según Pg.2 párrafo 3
        with tf.name_scope('preprocess') as scope:
            mean = tf.constant([123.68, 116.779, 103.939], dtype=tf.float32, shape=[1, 1, 1, 3], name='img_mean')
            images = self.imgs-mean

        #
        # Bloque 1
        # Cada bloque de una CNN está compuesto por 3 elementos:
        # 1. Convolución
        # 2. Activación
        # 3. Pooling
        #

        # Capa 1: conv1_1
        with tf.name_scope('conv1_1') as scope:
            # filtro de 3x3 (Pg. 2 párrafo 3)
            kernel = tf.Variable(tf.truncated_normal([3, 3, 3, 64], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            # [1, 1, 1, 1] - stride 1 (Pg. 2 párrafo 3)
            # padding='SAME' - bordes usan el mismo valor de sus vecinos
            conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
            # bias inicializados a 0 (Pg. 4 párrafo 4), no importa serán reescritos luego
            biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                 trainable=True, name='biases')
            # k*W + B
            out = tf.nn.bias_add(conv, biases)
            # activación ReLU para capas internas (Pg. 2, párrafo 5)
            self.conv1_1 = tf.nn.relu(out, name=scope)
            
            # añadimos a lista para facilitar inicialización de pesos
            self.parameters += [kernel, biases]

        # Capa 2: conv1_2
        with tf.name_scope('conv1_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 64], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv1_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv1_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # Pooling: pool1
        # max pooling 2x2, stride 2 (Pg. 2, párrafo 3)
        self.pool1 = tf.nn.max_pool(self.conv1_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool1')

        #
        # Bloque 2
        #

        # conv2_1
        with tf.name_scope('conv2_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 128], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv2_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv2_2
        with tf.name_scope('conv2_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 128], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv2_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv2_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool2
        self.pool2 = tf.nn.max_pool(self.conv2_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool2')

        #
        # Bloque 3
        #

        # conv3_1
        with tf.name_scope('conv3_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv3_2
        with tf.name_scope('conv3_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv3_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv3_3
        with tf.name_scope('conv3_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv3_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool3
        self.pool3 = tf.nn.max_pool(self.conv3_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool3')

        #
        # Bloque 4
        #

        # conv4_1
        with tf.name_scope('conv4_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool3, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv4_2
        with tf.name_scope('conv4_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv4_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv4_3
        with tf.name_scope('conv4_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv4_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool4
        self.pool4 = tf.nn.max_pool(self.conv4_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool4')

        #
        # Bloque 5
        #

        # conv5_1
        with tf.name_scope('conv5_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool4, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv5_2
        with tf.name_scope('conv5_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv5_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv5_3
        with tf.name_scope('conv5_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv5_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool5
        self.pool5 = tf.nn.max_pool(self.conv5_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool4')


    #
    # Capas Densas (fully connected)
    #
    def fc_layers(self):
        # fc1
        with tf.name_scope('fc1') as scope:
            shape = int(np.prod(self.pool5.get_shape()[1:]))
            # aplanar pesos
            fc1w = tf.Variable(tf.truncated_normal([shape, 4096],
                                                         dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
            # aplanar bias
            fc1b = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),
                                 trainable=True, name='biases')
            #
            pool5_flat = tf.reshape(self.pool5, [-1, shape])
            # convolución como producto de matrices
            fc1l = tf.nn.bias_add(tf.matmul(pool5_flat, fc1w), fc1b)
            self.fc1 = tf.nn.relu(fc1l)
            self.parameters += [fc1w, fc1b]

        # fc2
        with tf.name_scope('fc2') as scope:
            fc2w = tf.Variable(tf.truncated_normal([4096, 4096],
                                                         dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
            fc2b = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),
                                 trainable=True, name='biases')
            fc2l = tf.nn.bias_add(tf.matmul(self.fc1, fc2w), fc2b)
            self.fc2 = tf.nn.relu(fc2l)
            self.parameters += [fc2w, fc2b]

        # fc3
        # la última capa contiene 1000 elementos para 1000 clases
        with tf.name_scope('fc3') as scope:
            fc3w = tf.Variable(tf.truncated_normal([4096, 1000],
                                                         dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
            fc3b = tf.Variable(tf.constant(1.0, shape=[1000], dtype=tf.float32),
                                 trainable=True, name='biases')
            self.fc3l = tf.nn.bias_add(tf.matmul(self.fc2, fc3w), fc3b)
            self.parameters += [fc3w, fc3b]

    def load_weights(self, weight_file, sess):
        # carga de pesos del modelo usando numpy
        weights = np.load(weight_file)
        keys = sorted(weights.keys())
        for i, k in enumerate(keys):
            # print(i, k, np.shape(weights[k]))
            # asignar el peso a cada elemento de la lista (capas, bias)
            sess.run(self.parameters[i].assign(weights[k]))
            # print(self.parameters[i])

#### 3. Load the weights

In [3]:
sess = tf.Session()

# placeholder para la imagen de 224x244x3
imgs = tf.placeholder(tf.float32, [None, 224, 224, 3])

# construimos la red y cargamos los pesos
vgg = vgg16(imgs, 'vgg16_weights.npz', sess)


#### 4. Make a prediction

In [7]:
# hacemos una predicción 
img1 = imread('laska.png', mode='RGB')
img1 = imresize(img1, (224, 224))

# alimentamos el tensor con la imagen leída
# devuelve un vector de predicciones para cada clase
prob = sess.run(vgg.probs, feed_dict={vgg.imgs: [img1]})[0]

# ordenar predicción descendentemente y tomar las 5/1000
preds = (np.argsort(prob)[::-1])[0:5]
for p in preds:
    print("clase:#%d - probabilidad:%f" % (p, prob[p]))

clase:#356 - probabilidad:0.693385
clase:#358 - probabilidad:0.175389
clase:#357 - probabilidad:0.122085
clase:#359 - probabilidad:0.008871
clase:#360 - probabilidad:0.000121
